In [66]:
#Import all libraries needed for interface

#Dash modules for interface
import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import base64

#Pickle and Keras for model loading
from pickle import load
from keras.models import load_model

#Libraries for stock data and plotting
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import numpy as np
from datetime import timedelta
import datetime


#import models 
from forecast import Forecast

In [67]:
#Retrieve S&P 500 companies tickers and names
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df=table[0]
spCompanies=[]

for i in range(len(df)):
    companyInfo={'label': df['Security'][i],'value': df['Symbol'][i]}
    spCompanies.append(companyInfo)

In [68]:
app = dash.Dash()
model=Forecast()
image_filename = "stock.png" # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div(id = 'parent', style = {'backgroundColor' : '#202A44'}, 
                      children = [ 
    html.Img(src = 'stock.png'),
     
    html.H1('MCS13 Stock Analysis & Forecasting', 
            style = {'font-family' : 'monaco','font-weight' : 'bold', 'textAlign':'center', 'verticalAlign' : 'middle','marginTop':40,'marginBottom':40, 
                     'backgroundColor': "black", 'color': 'white'}),

        dcc.Dropdown( id = 'dropdown',
        options=spCompanies,
        value = 'AAPL'),
        
        html.Div([
            dcc.Interval(id="i_tris", interval=1*604800000, n_intervals=0),
            dcc.Interval(id="i_tris_m", interval=1*1800000, n_intervals=0),
            dcc.Interval(id="i_tris_d", interval=1*86400000, n_intervals=0),
            dcc.Interval(id="i_tris_h", interval=1*3600000, n_intervals=0),
            dcc.Graph(id = 'plot1', figure = {'layout': {'plotbgcolor': '#202A44', 'paperbgcolor':'#202A44'}},
                    config={
                        'doubleClick':'reset',
                        'showTips':True,
                        'displayModeBar':'hover',
                        'watermark':False})
            ]),


        html.Div([dcc.Graph(id = 'plot2', figure = {'layout': {'plotbgcolor': '#202A44', 'paperbgcolor':'#202A44'}})], 
                 style = {'width' : '33.33%','display' : 'inline-block'}),
        html.Div([dcc.Graph(id = 'plot3', figure = {'layout': {'plotbgcolor': '#202A44', 'paperbgcolor':'#202A44'}})], 
                 style = {'width' : '33.33%','display' : 'inline-block'}),
        html.Div([dcc.Graph(id = 'plot4', figure = {'layout': {'plotbgcolor': '#202A44', 'paperbgcolor':'#202A44'}})], 
                 style = {'width' : '33.33%','display' : 'inline-block'})
    ])



# Weekly Forecast
@app.callback(Output(component_id='plot1', component_property= 'figure'),
             [Input(component_id='dropdown', component_property= 'value'),
              Input(component_id="i_tris", component_property= "n_intervals")])

def graph_update1(dropdown_value, n):
    from forecast import Forecast
    model=Forecast()
    #get predicted price for next week
    stockPrices=yf.Ticker(dropdown_value).history('3mo',interval='1wk').filter(["Close"])
    predictedPrice=model.predict_next_week(stockPrices)

    #append price to previous week prices
    prices=yf.Ticker(dropdown_value).history(period='1mo', interval='1wk').filter(["Close"])
    date=prices.index[-1]
    


    newDate=date + timedelta(days=7)


    data={'Close': predictedPrice}
    prices= prices.append(pd.DataFrame(data, index=[newDate]))
    
    #Created frame to highlight predicted price
    predictDF=prices[-2:]

    # Drop last row
    prices.drop(index=prices.index[-1],axis=0,inplace=True)

    fig= px.line(prices,x=prices.index,y='Close',markers=True, template='plotly_dark')
    fig.update_traces(line_color='#FFFFFF', line_width=2)

    fig.add_scatter(x = predictDF.index, y = predictDF.Close, mode='lines',
                    line = dict(color = '#FF0000', width = 2),
                    marker = {'color': 'red'},
                    name = '',
                    showlegend=False)
    
    predictDF=predictDF[-1:]
    fig.add_scatter(x = predictDF.index, y = predictDF.Close,
                    marker = {'color': 'red'},
                    name = 'Predicted Price'
                    )
   
    # Created frame to highlight predicted price
    
    fig.update_layout(title ="Weekly Predicition",
                      xaxis_title = 'Date',
                      yaxis_title = 'Stock Prices'
                      )

    
    return fig

# Daily Forecast
@app.callback(Output(component_id='plot2', component_property= 'figure'),
             [Input(component_id='dropdown', component_property= 'value'),
             Input(component_id="i_tris_d", component_property= "n_intervals")])

def graph_update2(dropdown_value, n):
    
    #get predicted price for next week
    stockPrices=yf.Ticker(dropdown_value).history('6mo').filter(["Close"])
    predictedPrice=model.predict_next_day(stockPrices)


    #append price to previous week prices
    prices=yf.Ticker(dropdown_value).history(period='1mo').filter(["Close"])
    date=prices.index[-1]

    #Stock market is closed on weekends
    if date.weekday()>4:
        newDate=date + timedelta(days=3)
    else:
        newDate=date + timedelta(days=1)

    data={'Close': predictedPrice}
    prices= prices.append(pd.DataFrame(data, index=[newDate]))
    
    fig= px.line(prices,x=prices.index,y='Close',markers=True, template='plotly_dark')

     #Created frame to highlight predicted price
    predictDF=prices[-2:]

    # Drop last row
    prices.drop(index=prices.index[-1],axis=0,inplace=True)

    fig= px.line(prices,x=prices.index,y='Close',markers=True, template='plotly_dark')
    fig.update_traces(line_color='#FFFFFF', line_width=2)

    fig.add_scatter(x = predictDF.index, y = predictDF.Close, mode='lines',
                    line = dict(color = '#FF0000', width = 2),
                    marker = {'color': 'red'},
                    name = '',
                    showlegend=False)
    
    predictDF=predictDF[-1:]
    fig.add_scatter(x = predictDF.index, y = predictDF.Close,
                    marker = {'color': 'red'},
                    name = 'Predicted Price'
                    )

    fig.update_layout(title ="Daily Predicition",
                      xaxis_title = 'Date',
                      yaxis_title = 'Stock Prices'
                      )
    return fig

# Hourly Forecast
@app.callback(Output(component_id='plot3', component_property= 'figure'),
             [Input(component_id='dropdown', component_property= 'value'),
             Input(component_id="i_tris_h", component_property= "n_intervals")])

def graph_update3(dropdown_value, n):    
    #get predicted price for next week
    stockPrices=yf.Ticker(dropdown_value).history(period='5d',interval='1h').filter(["Close"])
    predictedPrice=model.predict_next_hour(stockPrices)


    #append price to previous week prices
    prices=yf.Ticker(dropdown_value).history(period='1d', interval='1h').filter(["Close"])
    date=prices.index[-1]
    newDate=date + timedelta(hours=1)

    data={'Close': predictedPrice}
    prices= prices.append(pd.DataFrame(data, index=[newDate]))
    
    fig= px.line(prices,x=prices.index,y='Close',markers=True,template='plotly_dark')
    

     #Created frame to highlight predicted price
    predictDF=prices[-2:]

    # Drop last row
    prices.drop(index=prices.index[-1],axis=0,inplace=True)

    fig= px.line(prices,x=prices.index,y='Close',markers=True, template='plotly_dark')
    fig.update_traces(line_color='#FFFFFF', line_width=2)

    fig.add_scatter(x = predictDF.index, y = predictDF.Close, mode='lines',
                    line = dict(color = '#FF0000', width = 2),
                    marker = {'color': 'red'},
                    name = '',
                    showlegend=False)
    
    predictDF=predictDF[-1:]
    fig.add_scatter(x = predictDF.index, y = predictDF.Close,
                    marker = {'color': 'red'},
                    name = 'Predicted Price'
                    )
    fig.update_layout(title ="Hourly Predicition",
                      xaxis_title = 'Date',
                      yaxis_title = 'Stock Prices'
                      )
    return fig

# 30 Minutes Forecast
@app.callback(Output(component_id='plot4', component_property= 'figure'),
             [Input(component_id='dropdown', component_property= 'value'),
             Input(component_id="i_tris_m", component_property= "n_intervals")])

def graph_update4(dropdown_value, n):
    
    #get predicted price for next week
    stockPrices=yf.Ticker(dropdown_value).history(period='5d', interval='30m').filter(["Close"])
    predictedPrice=model.predict_next_30m(stockPrices)

    print(predictedPrice)

    #append price to previous week prices
    prices=yf.Ticker(dropdown_value).history(period='1d', interval='30m').filter(["Close"])
    date=prices.index[-1]
    newDate=date + timedelta(minutes=30)

    data={'Close': predictedPrice}
    prices= prices.append(pd.DataFrame(data, index=[newDate]))
    
    fig= px.line(prices,x=prices.index,y='Close',markers=True,template='plotly_dark')
    

     #Created frame to highlight predicted price
    predictDF=prices[-2:]

    # Drop last row
    prices.drop(index=prices.index[-1],axis=0,inplace=True)

    fig= px.line(prices,x=prices.index,y='Close',markers=True, template='plotly_dark')
    fig.update_traces(line_color='#FFFFFF', line_width=2)

    fig.add_scatter(x = predictDF.index, y = predictDF.Close, mode='lines',
                    line = dict(color = '#FF0000', width = 2),
                    marker = {'color': 'red'},
                    name = '',
                    showlegend=False)
    
    predictDF=predictDF[-1:]
    fig.add_scatter(x = predictDF.index, y = predictDF.Close,
                    marker = {'color': 'red'},
                    name = 'Predicted Price'
                    )

    fig.update_layout(title ="30m predicition",
                      xaxis_title = 'Date',
                      yaxis_title = 'Stock Prices'
                      )

                      
    return fig


if __name__ == '__main__': 
    app.run_server(port=3004)

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is running on http://127.0.0.1:3004/

Dash is run

 * Running on http://127.0.0.1:3004/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_favicon.ico?v=2.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:17] "GET /stock.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 345ms/step
138.54343


C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:218: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:168: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [02/Oct/2022 16:40:18] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:120: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [02/Oct/2022 16:40:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:18] "POST /_dash-update-component HTTP/1.1" 200 -


1/1 [==============================] - 0s 122ms/step


...

C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:168: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [02/Oct/2022 16:40:46] "POST /_dash-update-component HTTP/1.1" 200 -


29.049671


C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:120: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:218: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [02/Oct/2022 16:40:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 16:40:47] "POST /_dash-update-component HTTP/1.1" 200 -


1/1 [==============================] - 0s 179ms/step


C:\Users\keong\AppData\Local\Temp\ipykernel_27512\1021471452.py:64: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [02/Oct/2022 16:40:48] "POST /_dash-update-component HTTP/1.1" 200 -
